In [20]:
import cv2
import os
import glob
import argparse
import pymc3 as pm
import numpy as np
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split

import theano 
import theano.tensor as T

In [21]:
def image_to_feature_vector(image, size=(12, 12)): return cv2.resize(image, size).flatten()

In [22]:
cats = map(image_to_feature_vector, [cv2.imread(cat,cv2.CV_LOAD_IMAGE_GRAYSCALE) for cat in glob.glob('data/cats/*')])
dogs = map(image_to_feature_vector, [cv2.imread(dog,cv2.CV_LOAD_IMAGE_GRAYSCALE) for dog in glob.glob('data/dogs/*')])

data = np.vstack((cats,dogs))
labels = np.array([1]*len(cats) + [0]*len(dogs))
data = np.array(data) / 255.0

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=.5)
ann_input = theano.shared(data)
ann_output = theano.shared(labels)

"""
init_1 = np.random.randn(data.shape[1], 768)
init_2 = np.random.randn(768, 384)
init_out = np.random.randn(384)
"""
init_1 = np.random.randn(data.shape[1], 5)
init_2 = np.random.randn(5, 5)
init_out = np.random.randn(5)

In [24]:
with pm.Model() as neural_network:
    # Weights from input to hidden layer
    W_in_1 = pm.Normal('w_in_1', 0, sd=1, 
                             shape=(data.shape[1], 5), # 768
                             testval=init_1)
    
    # Weights from 1st to 2nd layer
    W_1_2 = pm.Normal('w_1_2', 0, sd=1, 
                            shape=(5, 5), #shape=(768, 384), 
                            testval=init_2)
    
    # Weights from hidden lay2er to output
    W_2_out = pm.Normal('w_2_out', 0, sd=1, shape=(5,), testval=init_out) #shape=(384,)
    
    # Build neural-network using tanh activation function
    act_1 = T.nnet.relu(T.dot(ann_input, W_in_1))
    act_2 = T.nnet.relu(T.dot(act_1, W_1_2))
    act_out = T.nnet.sigmoid(T.dot(act_2, W_2_out))
    
    # Binary classification -> Bernoulli likelihood
    out = pm.Bernoulli('out', act_out, observed=ann_output)


In [25]:
print(neural_network)

In [26]:
from pymc3 import NUTS, sample,find_MAP,Slice,traceplot
from scipy import optimize


with neural_network:
    # Run ADVI which returns posterior means, standard deviations, and the evidence lower bound (ELBO)
    v_params = pm.variational.advi(n=1000)

"""
map_estimate = find_MAP(model=neural_network)

print(map_estimate)


with neural_network:
    # obtain starting values via MAP
    start = find_MAP(fmin=optimize.fmin_powell)

    # instantiate sampler
    step = Slice(vars=[sigma])

    # draw 5000 posterior samples
    trace = sample(5000, step=step, start=start)

"""


Average ELBO = -1,194.3: 100%|██████████| 1000/1000 [00:01<00:00, 995.18it/s]Finished [100%]: Average ELBO = -942.96


'\nmap_estimate = find_MAP(model=neural_network)\n\nprint(map_estimate)\n\n\nwith neural_network:\n    # obtain starting values via MAP\n    start = find_MAP(fmin=optimize.fmin_powell)\n\n    # instantiate sampler\n    step = Slice(vars=[sigma])\n\n    # draw 5000 posterior samples\n    trace = sample(5000, step=step, start=start)\n\n'

In [27]:
with neural_network:
    trace = pm.variational.sample_vp(v_params, draws=5001)


 27%|██▋       | 1327/5001 [00:00<00:00, 6671.09it/s]

100%|██████████| 5001/5001 [00:00<00:00, 5759.57it/s]

In [28]:
# Replace shared variables with testing set
ann_input.set_value(X_test)
ann_output.set_value(Y_test)

# Creater posterior predictive samples
ppc = pm.sample_ppc(trace, model=neural_network, samples=500)

# Use probability of > 0.5 to assume prediction of class 1
pred = ppc['out'].mean(axis=0) > 0.5

100%|██████████| 500/500 [00:03<00:00, 152.28it/s]


In [29]:
print('Accuracy = {}%'.format((Y_test == pred).mean() * 100))

Accuracy = 50.0%
